<a href="https://colab.research.google.com/github/radhika3131/Deep_Learning_with_Python/blob/main/ch_03_Getting_started_with_neural_network_Classification_and_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification and Regression glossary

classification and regression involves mnay specialized terms .

1. Sample or input- one data point that goes into your model
2. Prediction or output - what comes out of you model .
3. Target - The truth. what your model should ideally have predicted , according to an external source of data.
4. classes - A set of possible labels to chose from in a classification problem .For example , when  classifying cat and dog pictures , "dog" and "cat" are the two classes.
5. Label - A specific instance of class annotation in aclassification problem . For instance , if picture # 1234 is annotated as containing the class "dog" the "dog" is label of picture #1234
6. Griund-Truth or annotations - All targets for a dataset, typically collected by humans.
7. Binary Classification - A classification task where each input sample should be categorized into two exclusive categories.
8. Multilabel Classification - A classification task where each input samples can be asssigned multiple labels , For instance , a given image may contain both a cat and dog and should be annnotated both with the "cat" label and the "dog" label .**The number of labels per image is usually varible**.
9. Multiclass classification - A classification task where each input sample should be categorized into more than two categories:for instance , classifying handwritten digit.
10. Scalar Regression - A task where the target is a  continuous scalar value .Predicting house price is agood example: the different target price from continuous space.
11. Vector regression - a task where the target is a set of continuous values .for example , a continuous vector. if you are doing regression against  multiple values(such as coordinates of bounding box in a images.
12. Mini- batch or Batch - a small set of samples(typically betwenn 8 and 128)
that are processed simultaneously by the model . The number of samples is often a power of 2 , to facilitate the memory allocation of GPU . when training a mini-batch is used to ompute a single gradient descent update applied to the weights of the model


#Classifying movie reviews: a binary classification

you will learn to classify movie reviews as positive or negative ,based on the content of the movie

The IMDB dataset
 you will work work with the IMDB dataset : a set of 50,000 highly polarized reviews from the Internet Movie Database. They are split into 25,000 reviews for training and 25,000 reviews for testing , each set consisting of 50% negative and 50% positive reviews

Loading the dataset

In [1]:
from tensorflow.keras.datasets import imdb
(train_data , train_labels) , (test_data , test_labels) = imdb.load_data(num_words = 10000)
# the argument num_words = 100000 means you will only keep the top 10,000 most frequently occuring words in the training data, this allow us to work work with manageble size

17464789/17464789 [==============================] - 0s 0us/step


The variable train__data and test_data are list of reviews ;each review is a list of word indices.train_labels and test_labels are list of 0's and 1's , 0 stand for negatibe and 1 stand for positive

In [2]:
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [3]:
train_labels[0]

1

In [4]:
# as we are restricting ourselves on the top 10,000 nost frequent words , no word index will exceed 10,000
max([max(sequence) for sequence in train_data]) 

9999

Decoding reviews back to english word(text)

In [5]:
word_index = imdb.get_word_index() # word_index is a dictionary mapping words to an integer index
reverse_word_index = dict(
    [(value, key) for (key , value ) in word_index.items()]# reverse it map integer indexes to a word 
)

decode_review = " ".join(
    [reverse_word_index.get(i - 3 , "?") for i in train_data[0]]
)
# decode the review ,Note that the indices are offset by 3 becz 0 and 1 and 2 are reserved indices for "padding" , "start of sequence " and "unknown"

1641221/1641221 [==============================] - 0s 0us/step


#Preparing the data

you can directly feed the list into the neural network. they all have different length , but neural network expects to process contiguous batches of data, you have to turn your list into tensors

1. pad your list so that they all have the same length , turn them into a integer tensor of shaoe(samples, max_length )
2. Multi-hot encode your lists to turn them into vector of 0s and 1s

Encoding the integer sequence via multi-hot encoding

In [7]:
import numpy as np
def vectorize_sequences(sequences , dimension = 10000):
  results = np.zeros((len(sequences) , dimension)) # create an all -zeros matrix of shape(lrn(sequences), dimension)
  for i , sequence in enumerate(sequences):
    for j in sequence:
      results[i,j] = 1  # sets specific indices of results[i] to 1s
    return results

x_train = vectorize_sequences(train_data) # vectorized train data
x_test = vectorize_sequences(test_data) # vectorized test data

In [8]:
x_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

In [10]:
# you should also vectorized you labels
y_train = np.array(train_labels).astype("float32")
y_test = np.array(test_labels).astype("float32")

# now data is ready to fed into neural network

Building your model

A type of model that performs on such type of problem is a simple stack of densly connected(Dense) layer with relu activation function.
There are two key architecture decisions to be made about such a stack of Dense layers.
1. how many layers to chose
2. How many units to chose for each layer

Model definition

In [12]:
from tensorflow import keras 
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(16, activation = "relu"),
    layers.Dense(16, activation = "relu"),
    layers.Dense(1 , activation = "sigmoid")
])

Compiling the model

In [13]:
model.compile(
    optimizer = "rmsprop",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

# Validating your approach
its a standard practice to monitor the accuracy of the model during training .

Setting aside a validating set

In [14]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

Training the model

In [19]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/20


TypeError: ignored